### （為復現數據）製作 mom0 的流水線_2

湯底  && 目的 = 2_mom0/py_mom0_pipeline_oldData.ipynb  
但加入了轉換 flux 單位的部分 (Jy/beam -> K)  
媽的最好給我成功喔


In [23]:
from casatasks import importfits, immath, imhead, immoments, exportfits
import shutil
import os
import numpy as np

In [ ]:
'''
因為程式設計的問題所以很吃原生環境
我是說檔案位置的問題...這邊都是用 4545 的 beam 的
總之就是有這麼一回事
'''
commomPath = '/Users/aqing/Documents/1004/Circinus_galaxy/Ratio-Map/oldData/v2/4_brightnessTemp_ratio/smoothed-cube_copy/' # 從複製過的地方抓取 cube
beamL = [('3030', 1.1331*(np.pi*8.33e-4/180)**2), ('4545', 1.1331*(np.pi*1.25e-3/180)**2)] # (fileName, solidAngle)
mole = [('CO-1-0', 115.271), ('HCN-1-0', 88.63), ('CN-J12-12', 113.14), ('CN-J32-12', 113.49)] # (mole, restFrequency)

# costants
c = 3e8
k_B = 1.3807e-23
'''
其實 f_r 可以從 header 裡取，但這是因為這是處理優良的 cube。
像是 MVP 那種 ，一個 cube 裡面不只一條線的，就需要手動對 f_r 進行設定。
'''

'\n其實 f_r 可以從 header 裡取，但這是因為這是處理優良的 cube。\n像是 MVP 那種 ，一個 cube 裡面不只一條線的，就需要手動對 f_r 進行設定。\n'

In [ ]:
# 大巢

for f, freq in mole :
    print(f'Start making moment maps of {f}')

    for b, sr in beamL :
        pathIN = f'{commomPath}{f}_oldDataCube_smoothed-{b}.fits'
        importfits(fitsimage=pathIN, imagename='casaIN.im', overwrite=True)
        #print(f"Successfully import {f}'s cube (beam{b}) as a CASA file.")

        #print(f'Transforming flux unit of {f}')
        os.system("cp -r casaIN.im cp.im")
        scale = (c**2)/(2*k_B*sr*(freq**2))
        immath(imagename='cp.im', expr=f'{scale} * IM0', outfile='K.im')
        imhead(imagename='K.im', mode='put', hdkey='bunit', hdvalue='K') # 標示改個
        
        immoments(imagename='K.im', outfile='casaOUT.im', moments=[0], axis='spec')

        fitsOUT = f'{f}_oldData_mom0_smoothed-{b}_unit-K.fits'
        #print(f'Giving {fitsOUT}')
        exportfits(imagename='casaOUT.im', fitsimage=fitsOUT, overwrite=True)

        shutil.rmtree('casaOUT.im') # 雖然 overwrite=True，還是移一下
        shutil.rmtree('casaIN.im')
        shutil.rmtree('K.im')
        shutil.rmtree('cp.im')

    print(f'Moment maps of {f} are all done !')
    print()

print('FIN')

Start making moment maps of CO-1-0
Moment maps of CO-1-0 are all done !

Start making moment maps of HCN-1-0
Moment maps of HCN-1-0 are all done !

Start making moment maps of CN-J12-12
Moment maps of CN-J12-12 are all done !

Start making moment maps of CN-J32-12
Moment maps of CN-J32-12 are all done !

FIN
